In [1]:
# coding: utf-8
"""
英語コーパスを編集するためのコード
"""
from library.filer import Filer
import re
import glob
import random

1. Fileのパスを読み込み
2. Edmundsの2009年分のデータセットを全て読み込む(約150車種分)

In [3]:
list_path = glob.glob("../../../大学院/データセット/OpinRankDatasetWithJudgments/cars/data/2009/*")

list_all = []
for path in list_path:
    list_all.extend(Filer.readtxt(path))

1. 正規表現で<TEXT>の部分だけ読み込む
2. 先頭の<TEXT>、文末の</TEXT>を削除する
3. txt形式で保存する（約7000ユーザー分）

In [4]:
pattern = r'^<TEXT>'
list_all_rev = []
for row in list_all:
    if re.match(pattern, row):
        list_all_rev.append(row)
        
pattern1 = r'^<TEXT>'
pattern2 = r'</TEXT>'
list_all_rev1 = []
for row in list_all_rev:
    str1 = re.sub(pattern1, "", row)
    str2 = re.sub(pattern2, "", str1)
    list_all_rev1.append(str2)
    
Filer.writetxt(list_all_rev1, "../../../大学院/データセット/OpinRankDatasetWithJudgments/cars_all_review_2009.txt")

1. リストをシャッフルする
2. 1000ユーザー分だけを別のファイルに保存する

In [16]:
random.shuffle(list_all_rev1)

Filer.writetxt(list_all_rev1[0:1000], "../../../大学院/データセット/OpinRankDatasetWithJudgments/cars_all_review_2009_random1000.txt")

1. 1000ユーザー分のコーパスを読み込み
2. カンマで分割
3. 改行コードを削除する
4. tsvファイルにして保存

In [19]:
list_user = Filer.readtxt("../../../大学院/データセット/OpinRankDatasetWithJudgments/cars_all_review_2009_random1000.txt")

list_user_rev = []
for row in list_user:
    list_user_rev.append(row.split(". "))
    
list_user_rev1 = []
pattern = r"\r\n"
for row in list_user_rev:
    list_tmp = []
    for sentence in row:
        sentence1 = re.sub(pattern, "", sentence)
        if sentence1 != "":
            list_tmp.append(sentence1)
    list_user_rev1.append(list_tmp)
    
Filer.writetsv(list_user_rev1, "../../../大学院/データセット/OpinRankDatasetWithJudgments/cars_all_review_2009_random1000_sep.tsv")

### edmunds 評価用コーパスを抜き出す

1. 4人分のアノテーションを読み込む
2. エラーデータがないか確認
3. カンマで分割
4. 改行コードを削除する
5. tsvファイルにして保存

In [42]:
# 4人分のアノテーションを読み込む
list_csv_f1 = Filer.readcsv("./files/edmunds/annotation/edmunds_annotation_former_flavia.csv", option="rU")
list_csv_f2 = Filer.readcsv("./files/edmunds/annotation/edmunds_annotation_former_zhang.csv", option="rU")
list_csv_l1 = Filer.readcsv("./files/edmunds/annotation/edmunds_annotation_latter_danilo.csv", option="rU")
list_csv_l2 = Filer.readcsv("./files/edmunds/annotation/edmunds_annotation_latter_toshio.csv", option="rU")

# どのデータも破損していないか、確認
def check(list_csv, num=12):
    for i, row in enumerate(list_csv):
        if len(row) != num:
            print "Error:", i

check(list_csv_f1)
check(list_csv_f2)
check(list_csv_l1)
check(list_csv_l2)
        
# 文番号をintに、空白に0を"1"をintに変換する
def convert(list_csv):
    for i, row in enumerate(list_csv):
        list_csv[i][0] = int(list_csv[i][0])
        list_csv[i][1] = int(list_csv[i][1])
        for j in range(3,12):
            if list_csv[i][j] == "":
                list_csv[i][j] = 0
            else:
                list_csv[i][j] = 1
    return list_csv

list_csv_f1 = convert(list_csv_f1)
list_csv_f2 = convert(list_csv_f2)
list_csv_l1 = convert(list_csv_l1)
list_csv_l2 = convert(list_csv_l2)

# 2人の評価が同じものをデータセットとする
def match(list_csv1, list_csv2):
    list_csv_master = []
    for row1, row2 in zip(list_csv1, list_csv2):
        for i in range(3,11):
            if sum(row1[3:]) == 1 and sum(row2[3:]) == 1:
                if row1[i] == 1 and row2[i] == 1:
                    list_tmp = row1[:3]
                    list_tmp.append(i-3)
                    list_csv_master.append(list_tmp)
    return list_csv_master

list_csv_former = match(list_csv_f1, list_csv_f2)
list_csv_latter = match(list_csv_l1, list_csv_l2)
# 文番号でソート
list_csv_former = sorted(list_csv_former, key=lambda x:x[1])
list_csv_latter = sorted(list_csv_latter, key=lambda x:x[1])
# 1つのリストにまとめる
list_master = []
list_master.extend(list_csv_former)
list_master.extend(list_csv_latter)

In [51]:
list_master[7]

[10,
 67,
 "Truly, I thought I would give up a lot to save gas and the environment but instead was quite surprised! Got the hybrid fully loaded and couldn't be happier",
 0]

In [52]:
Filer.writetsv(list_master, "./files/edmunds/list_sentence_annotation.tsv")